In [1]:
DATAPATH="../datapath"
# 为测试集图片池和query文本计算特征
dataset_name="VQA"
split="test" # 指定计算valid或test集特征
resume=f"{DATAPATH}/experiments/vqa_finetune_vit-h-14_roberta-large_bs48_1gpu/checkpoints/epoch_latest.pt"

run_command = "export PYTHONPATH=${PYTHONPATH}:`pwd`/cn_clip;" + \
f"""
python -u cn_clip/eval/extract_features.py \
    --extract-image-feats \
    --extract-text-feats \
    --image-data="{DATAPATH}/datasets/{dataset_name}/lmdb/{split}/imgs" \
    --text-data="{DATAPATH}/datasets/{dataset_name}/{split}_texts.jsonl" \
    --img-batch-size=32 \
    --text-batch-size=32 \
    --context-length=52 \
    --resume={resume} \
    --vision-model=ViT-H-14 \
    --text-model=RoBERTa-wwm-ext-large-chinese
"""
print(run_command.lstrip())
!{run_command}

export PYTHONPATH=${PYTHONPATH}:`pwd`/cn_clip;
python -u cn_clip/eval/extract_features.py     --extract-image-feats     --extract-text-feats     --image-data="../datapath/datasets/VQA/lmdb/test/imgs"     --text-data="../datapath/datasets/VQA/test_texts.jsonl"     --img-batch-size=32     --text-batch-size=32     --context-length=52     --resume=../datapath/experiments/vqa_finetune_vit-h-14_roberta-large_bs48_1gpu/checkpoints/epoch_latest.pt     --vision-model=ViT-H-14     --text-model=RoBERTa-wwm-ext-large-chinese

Params:
  context_length: 52
  debug: False
  extract_image_feats: True
  extract_text_feats: True
  image_data: ../datapath/datasets/VQA/lmdb/test/imgs
  image_feat_output_path: None
  img_batch_size: 32
  precision: amp
  resume: ../datapath/experiments/vqa_finetune_vit-h-14_roberta-large_bs48_1gpu/checkpoints/epoch_latest.pt
  text_batch_size: 32
  text_data: ../datapath/datasets/VQA/test_texts.jsonl
  text_feat_output_path: None
  text_model: RoBERTa-wwm-ext-large-chinese

In [2]:
# 进行KNN检索，为测试集每个query，匹配特征余弦相似度最高的top-10商品图片
run_command = "export PYTHONPATH=${PYTHONPATH}:`pwd`/cn_clip;" + \
f"""
python -u cn_clip/eval/make_topk_predictions.py \
    --image-feats="{DATAPATH}/datasets/{dataset_name}/{split}_imgs.img_feat.jsonl" \
    --text-feats="{DATAPATH}/datasets/{dataset_name}/{split}_texts.txt_feat.jsonl" \
    --top-k=1 \
    --eval-batch-size=32768 \
    --output="{DATAPATH}/datasets/{dataset_name}/{split}_predictions.jsonl"
"""
print(run_command.lstrip())
!{run_command}

export PYTHONPATH=${PYTHONPATH}:`pwd`/cn_clip;
python -u cn_clip/eval/make_topk_predictions.py     --image-feats="../datapath/datasets/VQA/test_imgs.img_feat.jsonl"     --text-feats="../datapath/datasets/VQA/test_texts.txt_feat.jsonl"     --top-k=1     --eval-batch-size=32768     --output="../datapath/datasets/VQA/test_predictions.jsonl"

Params:
  eval_batch_size: 32768
  image_feats: ../datapath/datasets/VQA/test_imgs.img_feat.jsonl
  output: ../datapath/datasets/VQA/test_predictions.jsonl
  text_feats: ../datapath/datasets/VQA/test_texts.txt_feat.jsonl
  top_k: 1
Begin to load image features...
1884it [00:00, 3138.44it/s]
Finished loading image features.
Begin to compute top-1 predictions for texts...
392it [00:04, 82.94it/s] 
Top-1 predictions are saved in ../datapath/datasets/VQA/test_predictions.jsonl
Done!


In [3]:
import os
import re
import json


def match_first_text(query_text, test_texts):
    for test in test_texts:
        if query_text == test['text']:
            return test['text_id']
    return None


DATAPATH = "../datapath"
# 为测试集图片池和query文本计算特征
split = "test"  # 指定计算valid或test集特征

# query_path = f"{DATAPATH}/datasets/{dataset_name}/query.json"
query_path = f"{DATAPATH}/result/result_final1.json"
query_filter_path = f"{DATAPATH}/datasets/{dataset_name}/query_filter.json"
test_texts_path = f"{DATAPATH}/datasets/{dataset_name}/test_texts.jsonl"
output_path = f"{DATAPATH}/datasets/{dataset_name}/{split}_predictions.jsonl"
os.makedirs(f"{DATAPATH}/result", exist_ok=True)


test_texts = []
with open(test_texts_path, 'r') as f:
    for line in f:
        test_texts.append(json.loads(line))

# Read query.json
with open(query_path, 'r') as f:
    query_data = json.load(f)

# Extract relevant queries with the pattern "请匹配到与(.+?)最相关的图片"
query_filter = []
match_number = 0
for query in query_data:
    match = re.search(r'请匹配到与(.+?)最相关的图片', query["question"])
    if match:
        query_text = match.group(1).strip()
        text_id = match_first_text(query_text, test_texts)  # Find the first matching text_id
        match_number += 1
        # print(text_id)
        with open(output_path, 'r', encoding='utf-8') as f:
            for line in f:
                record = json.loads(line)
                if record["text_id"] == text_id:
                    target_image_id = record["image_ids"][0]
                    query["answer"] = target_image_id
                    break
    else:
        query_filter.append(query)

print(f'matched_queries={match_number}')

result_path = f"{DATAPATH}/result/result_update.json"
with open(result_path, 'w', encoding='utf-8') as json_file:
    json.dump(query_data, json_file, ensure_ascii=False, indent=4)

with open(query_filter_path, 'w', encoding='utf-8') as json_file:
    json.dump(query_filter, json_file, ensure_ascii=False, indent=4)



matched_queries=392


In [ ]:
import lmdb
import base64
from io import BytesIO
from PIL import Image

# 查看image
image_ids = ['gpgpovggzaeffilclosz.jpg']

lmdb_imgs = "../datapath/datasets/VQA/lmdb/train/imgs"
env_imgs = lmdb.open(lmdb_imgs, readonly=True, create=False, lock=False, readahead=False, meminit=False)
txn_imgs = env_imgs.begin(buffers=True)
for image_id in image_ids:
    image_b64 = txn_imgs.get("{}".format(image_id).encode('utf-8')).tobytes()
    img = Image.open(BytesIO(base64.urlsafe_b64decode(image_b64)))
    img.show()
